<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/KOSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KOSIS 홈페이지에서 생성한 Query를 그대로 가져와서 호출하는 예제

필요한 모듈을 임포트 한다.

In [30]:
import requests

KOSIS에서 복사해온 값을 Q에 넣는다. 너무 길면 맨 뒤에 \를 사용해서
여러 줄로 만들어 도 된다.


In [110]:
Q = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NTc5N2I2NWFlY2U2MWUyMTNhZTAxODc3NGVjN2JjM2Q=&itmId=T1+&objL1=ALL\
&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2000&endPrdDe=2023&orgId=101&tblId=DT_1B8000F"

API를 호출한다.

In [111]:
# API 요청
response = requests.get(Q)

반환값 중 JSON 부분만 추출한다.

In [112]:
json_data = response.json()

JSON을 출력해 본다.

In [113]:
# 응답 데이터 확인
print(json_data)

[{'TBL_NM': '인구동태건수 및 동태율 추이(출생사망혼인이혼)', 'PRD_DE': '2000', 'TBL_ID': 'DT_1B8000F', 'ITM_NM': '인구동태건수 및 동태율 추이', 'ITM_NM_ENG': 'Vital statistics(nubmer rate)', 'ITM_ID': 'T1', 'ORG_ID': '101', 'C1_OBJ_NM': '기본항목별', 'C1_OBJ_NM_ENG': 'By items', 'DT': '640089', 'PRD_SE': 'A', 'C1': '11', 'C1_NM': '출생아수(명)', 'C1_NM_ENG': 'Live births(persons)', 'LST_CHN_DE': '2018-07-03'}, {'TBL_NM': '인구동태건수 및 동태율 추이(출생사망혼인이혼)', 'PRD_DE': '2001', 'TBL_ID': 'DT_1B8000F', 'ITM_NM': '인구동태건수 및 동태율 추이', 'ITM_NM_ENG': 'Vital statistics(nubmer rate)', 'ITM_ID': 'T1', 'ORG_ID': '101', 'C1_OBJ_NM': '기본항목별', 'C1_OBJ_NM_ENG': 'By items', 'DT': '559934', 'PRD_SE': 'A', 'C1': '11', 'C1_NM': '출생아수(명)', 'C1_NM_ENG': 'Live births(persons)', 'LST_CHN_DE': '2018-07-03'}, {'TBL_NM': '인구동태건수 및 동태율 추이(출생사망혼인이혼)', 'PRD_DE': '2002', 'TBL_ID': 'DT_1B8000F', 'ITM_NM': '인구동태건수 및 동태율 추이', 'ITM_NM_ENG': 'Vital statistics(nubmer rate)', 'ITM_ID': 'T1', 'ORG_ID': '101', 'C1_OBJ_NM': '기본항목별', 'C1_OBJ_NM_ENG': 'By items', 'DT': '496911',

JSON 데이터는 간단히 Pandas DataFrame으로 변환할 수 있다.

In [114]:
import pandas as pd
df= pd.json_normalize(json_data)

변환한 데이터프레임을 출력해 보자.

In [ ]:
print(df)

무언가 이상하다. 2000년부터 2023년이면 24년치 데이터로서 행이 24개만 있을 줄 알았는데, 379개의 행이 있다.

전체에서 3개의 열('PRD_DE', 'DT', 'C1')만 추출해서 보자

In [116]:
df2 = df[['PRD_DE', 'DT', 'C1']]

나중에 연산을 위해 일단 모든 값이 수치 값이 되도록 하자.

In [117]:
df2.loc[:, 'DT'] = pd.to_numeric(df2['DT'], errors='coerce')
df2.loc[:, 'PRD_DE'] = pd.to_numeric(df2['PRD_DE'], errors='coerce')

맨 앞 30개만 출력해 보자.

In [ ]:
df2.head(30)

JSON의 중첩구조를 제대로 parsing하지 못해 여러개 열이 합쳐졌다.<BR>이를
분리해 보자.

우선 2023년부터 2000년까지가 중복되니까 분리할 개수를 계산하자.

In [119]:
# 행을 24개씩 자르고 이를 열로 변환
chunk_size = 2023 - 2000 +1

우선 chunk_size에 맞게 df를 분리해 f하나의 list로 만들어 보자.

In [120]:
chunks = [df2[i:i + chunk_size] for i in range(0, len(df2), chunk_size)]

chunks가 어떤 모습인지 출력해 보자

In [ ]:
print(chunks)

이제 chunks는 전체 df를 각각의 원소가 df인 여러 부분 df를 가진 하나의  긴 리스트로 만들었다.

이제 이 여러 df의 리스트를 하나의 DataFrame으로 결합하자. 새로운 DF의 인덱스는<BR>
연도로 설정한다.

In [122]:
# 각 조각을 하나의 DataFrame으로 결합
df_transformed = pd.concat([chunk.set_index('PRD_DE').reset_index(drop=True) for chunk in chunks], axis=1)

변환된 df를 출력해 보자.

In [ ]:
print(df_transformed)

열이름이 중복해서 들어갔으니, 각 열이름뒤에 일련번호를 증가시켜 다른 이름을 부여한다.

In [ ]:
# 열 이름 재정의
new_columns = []
for i, col in enumerate(df_transformed.columns):
    chunk_num = (i // (2 * chunk_size // chunk_size)) + 1  # 각 그룹은 2개의 열(DT, CI)로 구성됨
    col_type = 'DT' if 'DT' in col else 'CI'  # 'DT'와 'CI' 구분
    new_columns.append(f'{col_type}_{chunk_num}')

새로 생성한 이름 규칙에 따른 리스트를 df_transformed 열 이름으로 지정하자.

In [ ]:
df_transformed.columns = new_columns

인덱스는 PRD_DE의 연도로 설정하자.

In [ ]:
# 인덱스 재설정
df_transformed.index = df2['PRD_DE'][:chunk_size].values

최종적으로 변환된 데이터프레임을 출력해 보자.

In [ ]:
print(df_transformed)

최종 데이터프레임은 Pandas의 기본 함수를 이용해 쉽게 도식화 해 볼 수 있다.
<BR>우선 2000년 부터 2023년까지 해마다 태아난 신생아 수 변화를 그래프로 그려보자.

In [ ]:
df_transformed.DT_1.plot()